# ばんえい競馬予想アプリ - 当日予測

**最終更新: 2026-01-26**

---

**ばんえい競馬（帯広）専用の予測ノートブック**

## ばんえい競馬の特性

- **コース**: 直線200m（帯広競馬場のみ）
- **馬体重**: 800〜1200kg（通常競馬の約2倍）
- **ソリ重量**: 480〜1000kg（斤量の代わり）
- **障害**: 2つの坂を越える
- **騎手影響度**: 馬3:騎手7（通常競馬は馬7:騎手3）
- **馬場状態**: 水分量（%）で表示

## 使用方法
1. 左サイドバーの🔑からシークレットに以下を設定：
   - `SUPABASE_KEY`
2. セルを順番に実行（セクション1〜6）
3. セクション7でレース番号を指定して予測実行

## 1. 環境セットアップ

In [ ]:
# 必要なパッケージをインストール
!pip install -q supabase requests beautifulsoup4 lxml pandas numpy lightgbm scikit-learn

In [ ]:
# Supabase認証情報の設定
try:
    from google.colab import userdata
    SUPABASE_URL = userdata.get('SUPABASE_URL') or "https://khpoovshkhppadsirgcp.supabase.co"
    SUPABASE_KEY = userdata.get('SUPABASE_KEY')
    if SUPABASE_KEY:
        print("✓ Colabシークレットから認証情報を取得しました")
    else:
        raise Exception("SUPABASE_KEY not set")
except:
    # 直接入力（テスト用）
    SUPABASE_URL = "https://khpoovshkhppadsirgcp.supabase.co"  # @param {type:"string"}
    SUPABASE_KEY = "your-service-role-key"  # @param {type:"string"}
    print("⚠ 手動入力の認証情報を使用します")

# 接続テスト
from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print(f"✓ Supabase接続: {SUPABASE_URL[:40]}...")

## 2. モデルのダウンロード

In [ ]:
import pickle
import io
import pandas as pd
import numpy as np

BUCKET_NAME = "models"
RACE_TYPE = "banei"  # ばんえい競馬

def list_available_models():
    """利用可能なモデル一覧を取得"""
    try:
        files = supabase.storage.from_(BUCKET_NAME).list()
        # ばんえい用モデルのみフィルタ
        models = [f["name"] for f in files if f["name"].endswith(".pkl") and "banei" in f["name"]]
        print(f"利用可能なばんえいモデル: {models}")
        return models
    except Exception as e:
        print(f"エラー: {e}")
        return []

def download_model(version: str = "v1"):
    """ばんえい用モデルをダウンロード"""
    filename = f"model_banei_{version}.pkl"
    print(f"モデルをダウンロード中: {filename}")

    try:
        response = supabase.storage.from_(BUCKET_NAME).download(filename)
        buffer = io.BytesIO(response)
        model_data = pickle.load(buffer)
        print(f"✓ モデルをダウンロードしました")
        print(f"  バージョン: {model_data.get('model_version', 'unknown')}")
        print(f"  特徴量数: {len(model_data.get('feature_columns', []))}")
        return model_data
    except Exception as e:
        print(f"✗ ダウンロード失敗: {e}")
        return None

# 利用可能なモデル一覧を表示
list_available_models()

In [ ]:
# モデルをダウンロード
MODEL_VERSION = "v20260126_105855"  # @param {type:"string"}

model_data = download_model(MODEL_VERSION)

if model_data:
    model = model_data["model"]
    scaler = model_data["scaler"]
    calibrator = model_data.get("calibrator")
    feature_columns = model_data["feature_columns"]
    print(f"\n✓ モデル準備完了（{len(feature_columns)}特徴量）")
    print(f"\n特徴量一覧（先頭10個）:")
    for col in feature_columns[:10]:
        print(f"  - {col}")
else:
    print("\n✗ モデルのダウンロードに失敗しました")
    print("ローカルFastAPIでばんえい用モデルを学習・アップロードしてください")

## 3. 当日レースデータの取得（ばんえい競馬）

In [ ]:
import requests
import time
import re
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
from typing import Optional, List, Dict

# ブラウザ自動化設定（Colab用）
USE_BROWSER = True  # @param {type:"boolean"} ブラウザでオッズを取得

if USE_BROWSER:
    print("ブラウザ環境をセットアップ中...")
    
    # google-colab-selenium: Colab専用パッケージ
    !pip install -q google-colab-selenium
    
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import google_colab_selenium as gs
    
    def get_chrome_driver():
        """Colab用Chrome WebDriverを取得"""
        options = Options()
        options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("--window-size=1920,1080")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
        
        driver = gs.Chrome(options=options)
        return driver
    
    # 動作確認
    try:
        print("  ChromeDriverをセットアップ中...")
        test_driver = get_chrome_driver()
        test_driver.get("https://www.google.com")
        title = test_driver.title
        test_driver.quit()
        print(f"✓ ブラウザ準備完了（テストページ: {title}）")
    except Exception as e:
        print(f"⚠ ブラウザ初期化エラー: {e}")
        USE_BROWSER = False

# スクレイピング設定
SCRAPE_INTERVAL = 1.5
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

# ばんえい競馬場コード
BANEI_COURSE_CODE = "65"  # 帯広

session = requests.Session()
session.headers.update(HEADERS)
last_request_time = 0

def fetch_html(url: str) -> str:
    global last_request_time
    elapsed = time.time() - last_request_time
    if elapsed < SCRAPE_INTERVAL:
        time.sleep(SCRAPE_INTERVAL - elapsed)

    response = session.get(url, timeout=30)
    last_request_time = time.time()

    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"
    else:
        response.encoding = response.apparent_encoding or "utf-8"

    return response.text

def is_banei_race(race_id: str) -> bool:
    """ばんえい競馬かどうか判定"""
    if len(race_id) >= 6:
        return race_id[4:6] == BANEI_COURSE_CODE
    return False

print("✓ スクレイピング関数を定義しました")

In [ ]:
def scrape_race_list_banei(target_date: date) -> List[Dict]:
    """指定日のばんえい競馬レース一覧を取得（nar.netkeiba.com）"""
    date_str = target_date.strftime("%Y%m%d")
    url = f"https://nar.netkeiba.com/top/race_list_sub.html?kaisai_date={date_str}"

    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")

    races = []
    seen_ids = set()

    for link in soup.find_all("a", href=True):
        href = link.get("href", "")
        match = re.search(r"race_id=(\d{12})", href)
        if match:
            race_id = match.group(1)
            if race_id not in seen_ids:
                # ばんえい競馬のみ
                if not is_banei_race(race_id):
                    continue
                seen_ids.add(race_id)
                races.append({
                    "race_id": race_id,
                    "date": target_date.isoformat(),
                    "race_name": link.get_text(strip=True),
                    "course": "帯広",
                })

    races.sort(key=lambda x: x["race_id"])
    return races


def get_odds_with_selenium_banei(race_id: str, driver, max_retries: int = 3, expected_count: int = 0) -> Dict[int, Dict]:
    """Seleniumでばんえい競馬のオッズを取得"""
    url = f"https://nar.netkeiba.com/odds/index.html?race_id={race_id}&type=b1"
    best_result = {}

    for attempt in range(max_retries):
        try:
            driver.get(url)
            try:
                WebDriverWait(driver, 20).until(
                    lambda d: len(d.find_elements(By.CSS_SELECTOR, "#odds_tan_block td.Odds span")) > 0
                )
            except:
                time.sleep(3.0)

            time.sleep(2.5)

            result = {}
            rows = driver.find_elements(By.CSS_SELECTOR, "#odds_tan_block tr")

            for row in rows:
                try:
                    waku_elems = row.find_elements(By.CSS_SELECTOR, "td[class^='Waku']")
                    if not waku_elems:
                        continue
                    umaban_text = waku_elems[0].text.strip()
                    if not umaban_text.isdigit():
                        continue
                    umaban = int(umaban_text)

                    odds_elem = row.find_elements(By.CSS_SELECTOR, "td.Odds span")
                    if odds_elem:
                        odds_text = odds_elem[0].text.strip()
                        if odds_text and odds_text not in ["---.-", "---", ""]:
                            result[umaban] = {
                                "odds": float(odds_text),
                                "popularity": None,
                            }
                except:
                    continue

            if len(result) > len(best_result):
                best_result = result.copy()

            if result and (expected_count == 0 or len(result) >= expected_count * 0.8):
                return result

            if attempt < max_retries - 1:
                time.sleep(2.5)

        except Exception as e:
            if attempt < max_retries - 1:
                print(f"  ⚠ オッズ取得エラー、リトライ: {e}")
                time.sleep(2.5)

    return best_result


def scrape_shutuba_banei(race_id: str, driver=None) -> Dict:
    """ばんえい競馬の出馬表を取得"""
    url = f"https://nar.netkeiba.com/race/shutuba.html?race_id={race_id}"

    response = session.get(url, timeout=30)
    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"

    soup = BeautifulSoup(response.text, "lxml")

    race_data = {
        "race_id": race_id,
        "course": "帯広",
        "race_number": int(race_id[10:12]) if len(race_id) >= 12 else 0,
        "track_type": "ばんえい",
        "distance": 200,  # ばんえいは200m固定
    }

    # レース名
    title_elem = soup.select_one(".RaceName")
    if title_elem:
        race_data["race_name"] = title_elem.get_text(strip=True)

    # 馬場状態（水分量）
    race_data_elem = soup.select_one(".RaceData01")
    if race_data_elem:
        text = race_data_elem.get_text()
        # 水分量を抽出（例: "2.3%"）
        moisture_match = re.search(r"(\d+\.?\d*)%", text)
        if moisture_match:
            race_data["moisture"] = float(moisture_match.group(1))

    # グレード
    grade_elem = soup.select_one(".RaceData02 span")
    if grade_elem:
        race_data["grade"] = grade_elem.get_text(strip=True)

    # 出走馬を取得
    entries = []
    for row in soup.select("tr.HorseList"):
        entry = parse_shutuba_row_banei(row)
        if entry:
            entries.append(entry)

    field_size = len(entries)

    # オッズを取得
    odds_data = {}
    if USE_BROWSER and driver:
        odds_data = get_odds_with_selenium_banei(race_id, driver, expected_count=field_size)
        if odds_data:
            print(f"  ✓ {len(odds_data)}頭のオッズを取得")
        else:
            print(f"  ⚠ オッズ取得失敗 - デフォルト値を使用")

    # オッズをマージ
    for entry in entries:
        horse_num = entry.get("horse_number")
        if horse_num and horse_num in odds_data:
            entry["odds"] = odds_data[horse_num].get("odds")
            entry["popularity"] = odds_data[horse_num].get("popularity")

    race_data["entries"] = entries
    race_data["field_size"] = field_size
    return race_data


def parse_shutuba_row_banei(row) -> Optional[Dict]:
    """ばんえい競馬の出馬表行をパース"""
    entry = {}

    tds = row.find_all("td")

    if len(tds) > 0:
        try:
            entry["frame_number"] = int(tds[0].get_text(strip=True))
        except:
            pass

    if len(tds) > 1:
        try:
            entry["horse_number"] = int(tds[1].get_text(strip=True))
        except:
            return None

    horse_link = row.select_one("span.HorseName a")
    if horse_link:
        href = horse_link.get("href", "")
        match = re.search(r"/horse/(\d+)", href)
        if match:
            entry["horse_id"] = match.group(1)
        entry["horse_name"] = horse_link.get_text(strip=True)

    jockey_td = row.select_one("td.Jockey a")
    if jockey_td:
        href = jockey_td.get("href", "")
        match = re.search(r"/jockey/(?:result/recent/)?([a-zA-Z0-9]+)", href)
        if match:
            entry["jockey_id"] = match.group(1)
        entry["jockey_name"] = jockey_td.get_text(strip=True)

    # ソリ重量（ばんえい特有）
    if len(tds) > 5:
        try:
            entry["sori_weight"] = float(tds[5].get_text(strip=True))
        except:
            pass

    # 馬体重
    weight_td = row.select_one("td.Weight")
    if weight_td:
        try:
            weight_text = weight_td.get_text(strip=True)
            weight_match = re.search(r"(\d+)", weight_text)
            if weight_match:
                entry["horse_weight"] = int(weight_match.group(1))
            # 体重増減
            diff_match = re.search(r"\(([+-]?\d+)\)", weight_text)
            if diff_match:
                entry["weight_diff"] = int(diff_match.group(1))
        except:
            pass

    return entry if entry.get("horse_number") else None


print("✓ ばんえい競馬スクレイピング関数を定義しました")

In [ ]:
# 対象日を設定
target_date = date.today()  # 今日
# target_date = date(2025, 1, 26)  # 特定の日付

print(f"対象日: {target_date}")

# レース一覧を取得
races = scrape_race_list_banei(target_date)
print(f"\n{len(races)}件のばんえい競馬レースが見つかりました")

if races:
    print("\n【帯広競馬場】")
    for race in races:
        race_num = int(race["race_id"][10:12])
        print(f"  {race_num}R: {race['race_name']}")
else:
    print("\n本日はばんえい競馬の開催がないようです。")
    print("別の日付を指定してください。")

## 4. 特徴量の生成（ばんえい専用）

In [ ]:
# ばんえい用グレードマッピング
BANEI_GRADE_MAP = {
    "新馬": 0, "未勝利": 1,
    "C4": 2, "C3": 3, "C2": 4, "C1": 5,
    "B4": 6, "B3": 7, "B2": 8, "B1": 9,
    "A2": 10, "A1": 11,
    "オープン": 12, "OP": 12,
    "BG3": 13, "BG2": 14, "BG1": 15,
    "特別": 11,
}

# 性別マッピング
SEX_MAP = {"牡": 0, "牝": 1, "セ": 2}


def create_features_banei(race_data: Dict) -> pd.DataFrame:
    """
    ばんえい競馬用の特徴量を生成
    """
    entries = race_data.get("entries", [])
    if not entries:
        return pd.DataFrame()

    features_list = []
    field_size = race_data.get("field_size", len(entries))
    moisture = race_data.get("moisture", 2.0)  # デフォルト水分量
    grade = race_data.get("grade", "")
    grade_encoded = BANEI_GRADE_MAP.get(grade, -1)

    # ソリ重量の平均を計算
    sori_weights = [e.get("sori_weight", 0) for e in entries if e.get("sori_weight")]
    avg_sori_weight = np.mean(sori_weights) if sori_weights else 0

    # 季節特徴量
    race_date = date.today()
    month = race_date.month
    if month in [3, 4, 5]:
        season = 1  # 春
    elif month in [6, 7, 8]:
        season = 2  # 夏
    elif month in [9, 10, 11]:
        season = 3  # 秋
    else:
        season = 0  # 冬
    is_cold_season = 1 if month in [11, 12, 1, 2, 3] else 0

    for i, entry in enumerate(entries):
        sori_weight = entry.get("sori_weight", 0) or 0
        horse_weight = entry.get("horse_weight", 900) or 900
        odds = entry.get("odds", 10) or 10

        # 基本特徴量（モデルの特徴量に合わせる）
        features = {
            "horse_number": entry.get("horse_number"),
            
            # ID特徴量
            "horse_id_int": int(entry.get("horse_id", "0")) if entry.get("horse_id", "").isdigit() else 0,
            "jockey_id_int": int(entry.get("jockey_id", "0")) if entry.get("jockey_id", "").isdigit() else 0,
            "trainer_id_int": 0,  # 出馬表からは取得不可
            "umaban": entry.get("horse_number", 0),
            
            # ソリ重量関連
            "sori_weight": sori_weight,
            "sori_weight_ratio": sori_weight / horse_weight if horse_weight > 0 else 0,
            "sori_weight_rank": 0,  # 後で計算
            "sori_weight_normalized": (sori_weight - avg_sori_weight) / 100 if avg_sori_weight > 0 else 0,
            "sori_weight_vs_class_avg": sori_weight - (750 if "A" in grade else 680 if "B" in grade else 620 if "C" in grade else 580),
            
            # 馬体重関連
            "horse_weight_banei": horse_weight,
            "weight_diff_banei": entry.get("weight_diff", 0) or 0,
            "weight_trend": 0,  # 過去データがないため0
            "power_index": horse_weight / (sori_weight + 1) if sori_weight > 0 else 0,
            "optimal_weight_gap": 0,  # 過去データがないため0
            
            # 水分量・馬場
            "moisture_level": moisture,
            "moisture_aptitude": 0,  # 過去データがないため0
            "is_light_track": 1 if moisture < 2.0 else 0,
            "is_heavy_track": 1 if moisture > 4.0 else 0,
            
            # レース条件
            "grade_encoded": grade_encoded,
            "race_number": race_data.get("race_number", 1),
            "field_size": field_size,
            "frame_number": entry.get("frame_number", 0) or 0,
            
            # 馬基本情報（出馬表からは取得困難なのでデフォルト）
            "age": 5,  # デフォルト
            "age_category": 1,  # 中堅
            "sex": 0,  # 牡
            "sex_weight_bonus": 0,
            
            # 騎手（過去データがないためデフォルト）
            "jockey_win_rate": 0.1,
            "jockey_place_rate": 0.2,
            "jockey_year_rank": 10,
            "jockey_heavy_win_rate": 0.1,
            "jockey_moisture_apt": 0.2,
            "jockey_horse_combo": 0,
            
            # 過去成績（過去データがないためデフォルト）
            "avg_rank_last3": 5,
            "avg_rank_last5": 5,
            "win_rate": 0.1,
            "place_rate": 0.2,
            "show_rate": 0.3,
            "days_since_last": 14,
            "last_result": 5,
            "best_rank": 3,
            
            # 重量別成績
            "light_weight_win_rate": 0.1,
            "mid_weight_win_rate": 0.1,
            "heavy_weight_win_rate": 0.1,
            
            # オッズ
            "odds": odds,
            "log_odds": np.log1p(odds),
            "popularity": entry.get("popularity", 5) or 5,
            
            # 季節
            "month": month,
            "season": season,
            "is_cold_season": is_cold_season,
        }

        features_list.append(features)

    df = pd.DataFrame(features_list)

    # ソリ重量順位を計算
    df["sori_weight_rank"] = df["sori_weight"].rank(ascending=False).astype(int)

    # 足りない特徴量を0で埋める
    for col in feature_columns:
        if col not in df.columns:
            df[col] = 0

    return df

print("✓ ばんえい特徴量生成関数を定義しました")

## 5. 予測の実行

In [ ]:
def predict_race_banei(race_data: Dict) -> Dict:
    """ばんえい競馬レースの予測を実行"""
    df = create_features_banei(race_data)

    if df.empty:
        return None

    # 特徴量を選択
    X = df[feature_columns].copy()
    X = X.fillna(0)

    # スケーリング
    X_scaled = pd.DataFrame(
        scaler.transform(X),
        columns=X.columns,
        index=X.index,
    )

    # 予測
    probs = model.predict(X_scaled)

    # 確率の正規化
    total = probs.sum()
    if total > 0:
        probabilities = probs / total
    else:
        probabilities = probs

    # 結果を整形
    results = []
    entries = race_data.get("entries", [])

    for i, entry in enumerate(entries):
        odds = entry.get("odds") or 10
        prob = float(probabilities[i])
        ev = prob * odds

        results.append({
            "horse_number": entry.get("horse_number"),
            "horse_name": entry.get("horse_name", "不明"),
            "jockey_name": entry.get("jockey_name", ""),
            "sori_weight": entry.get("sori_weight"),
            "horse_weight": entry.get("horse_weight"),
            "score": float(probs[i]),
            "probability": prob,
            "odds": odds,
            "expected_value": ev,
            "popularity": entry.get("popularity"),
        })

    # スコア順にソート
    results.sort(key=lambda x: x["score"], reverse=True)

    # 順位付け
    for i, r in enumerate(results):
        r["pred_rank"] = i + 1

    return {
        "race_id": race_data["race_id"],
        "race_name": race_data.get("race_name", ""),
        "course": race_data.get("course", "帯広"),
        "race_number": race_data.get("race_number"),
        "moisture": race_data.get("moisture"),
        "grade": race_data.get("grade"),
        "predictions": results,
    }

print("✓ 予測関数を定義しました")

## 6. 個別レース予測

In [ ]:
# レース一覧を表示
import unicodedata

def get_east_asian_width(text):
    """文字列の表示幅を計算（全角=2, 半角=1）"""
    width = 0
    for char in str(text):
        if unicodedata.east_asian_width(char) in 'FWA':
            width += 2
        else:
            width += 1
    return width

def pad_to_width(text, width):
    """指定幅になるようにパディング"""
    text = str(text)
    current_width = get_east_asian_width(text)
    padding = width - current_width
    return text + ' ' * max(0, padding)

print("=== 本日のばんえいレース一覧 ===")
print()
print(f"  No.  場所      レース名")
print("-" * 50)
for i, race in enumerate(races):
    race_num = int(race["race_id"][10:12])
    race_name = race['race_name'][:15] if len(race['race_name']) > 15 else race['race_name']
    name_padded = pad_to_width(race_name, 30)
    print(f"  {i+1:2d}.  帯広{race_num:2d}R  {name_padded}  (ID: {race['race_id']})")

In [ ]:
# 個別レース予測

import unicodedata

def get_east_asian_width(text):
    """文字列の表示幅を計算（全角=2, 半角=1）"""
    width = 0
    for char in str(text):
        if unicodedata.east_asian_width(char) in 'FWA':
            width += 2
        else:
            width += 1
    return width

def pad_to_width(text, width):
    """指定幅になるようにパディング"""
    text = str(text)
    current_width = get_east_asian_width(text)
    padding = width - current_width
    return text + ' ' * max(0, padding)

# 推奨馬のフィルター条件
EV_THRESHOLD = 1.0   # 期待値下限
MAX_EV = 2.0         # 期待値上限（高すぎる穴馬を除外）
MIN_PROB = 0.05      # 最低確率5%
TOP_N_PER_RACE = 3   # レースごとの最大推奨数

# 上の一覧から番号を指定するか、race_idを直接入力
RACE_INDEX = 1  # @param {type:"integer"} 一覧の番号（1始まり）
# または直接race_idを指定（空欄の場合は上の番号を使用）
RACE_ID = ""  # @param {type:"string"}

# WebDriverを初期化
driver = None
if USE_BROWSER:
    try:
        driver = get_chrome_driver()
    except Exception as e:
        print(f"⚠ ブラウザ初期化エラー: {e}")

# レースを特定
if RACE_ID:
    target_race_id = RACE_ID
else:
    if races and 1 <= RACE_INDEX <= len(races):
        target_race_id = races[RACE_INDEX - 1]["race_id"]
    else:
        print(f"エラー: 番号は1〜{len(races) if races else 0}の範囲で指定してください")
        target_race_id = None

if target_race_id:
    print(f"予測対象: {target_race_id}")
    print(f"現在時刻: {datetime.now().strftime('%H:%M:%S')}")
    print("-" * 70)
    
    try:
        # 出馬表とオッズを取得
        print("\n出馬表とオッズを取得中...")
        race_data = scrape_shutuba_banei(target_race_id, driver=driver)
        
        # オッズから人気を算出
        entries_with_odds = [(i, e.get("odds", 999)) for i, e in enumerate(race_data.get("entries", []))]
        entries_with_odds.sort(key=lambda x: x[1])
        for rank, (idx, _) in enumerate(entries_with_odds, 1):
            race_data["entries"][idx]["popularity"] = rank
        
        # 予測実行
        pred = predict_race_banei(race_data)
        
        if pred:
            # 結果表示
            moisture = pred.get('moisture')
            moisture_str = f"{moisture}%" if moisture is not None else "-"
            
            print(f"\n{'='*70}")
            print(f"帯広 {pred.get('race_number', '')}R {pred['race_name']}")
            print(f"グレード: {pred.get('grade') or '-'}  水分量: {moisture_str}")
            print(f"{'='*70}")
            print(f"  予想  馬番  馬名                  勝率    オッズ  期待値  人気")
            print("-" * 70)
            
            for p in pred["predictions"]:
                prob_pct = (p["probability"] or 0) * 100
                odds = p.get('odds')
                odds_str = f"{odds:.1f}" if odds else "-"
                ev = p.get("expected_value") or 0
                pop = p.get("popularity") or "-"
                name_padded = pad_to_width(p['horse_name'], 18)
                
                # 推奨マーク
                if EV_THRESHOLD <= ev <= MAX_EV and (p.get("probability") or 0) >= MIN_PROB:
                    mark = "★"
                else:
                    mark = ""
                
                print(f"  {p['pred_rank']:>4}  {p['horse_number']:>4}  {name_padded}  {prob_pct:>5.1f}%  {odds_str:>6}  {ev:>5.2f}{mark:<2} {pop:>4}")
            
            # 推奨馬のみ抽出
            recommended = [p for p in pred["predictions"] 
                           if EV_THRESHOLD <= (p.get("expected_value") or 0) <= MAX_EV 
                           and (p.get("probability") or 0) >= MIN_PROB]
            if recommended:
                print(f"\n【推奨馬（EV {EV_THRESHOLD}〜{MAX_EV}、確率{MIN_PROB*100:.0f}%以上）】")
                for p in sorted(recommended, key=lambda x: x["expected_value"], reverse=True)[:TOP_N_PER_RACE]:
                    print(f"  馬番{p['horse_number']:2d} {p['horse_name']}: EV={p['expected_value']:.2f}, オッズ={p['odds']:.1f}")
        else:
            print("予測に失敗しました")
    
    except Exception as e:
        print(f"エラー: {e}")
    finally:
        if driver:
            driver.quit()

---

## 使い方まとめ

### 事前準備（ローカルPCで実行）

1. **ばんえい用モデル学習**
   - フロントエンドのばんえい > モデル管理画面で「再学習を開始」
   - または API: `POST /api/v1/model/retrain` with `{"race_type": "banei", ...}`

2. **モデルをSupabase Storageにアップロード**
   - `POST /api/v1/model/storage/upload?race_type=banei`

### 当日予測（このノートブック）

1. セル1〜5を順番に実行（環境セットアップ〜予測関数定義）
2. セル6のレース一覧で対象レースの番号を確認
3. `RACE_INDEX` を変更して予測実行

### ばんえい競馬のポイント

- **ソリ重量**: 重いほど不利だが、強い馬は重いソリでも勝つ
- **水分量**: 馬場の水分量（%）が重要、軽い馬場と重い馬場で得意不得意がある
- **騎手**: 通常競馬より騎手の影響度が高い（馬3:騎手7）
- **季節**: 冬季（11月〜3月）は馬場状態が大きく変わる

### 開催日程

- 帯広競馬場で週4〜5日開催
- 主に土日月+平日1〜2日